### 对数据样本进行特征转换的降维
常见的降维包括：独立成分分析ICA、主成分分析PCA、因子分析FA、线性判别分析LDA、局部线性嵌入LLE、核主成分分析Kernel PCA等

In [2]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import feature_selection
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import PolynomialFeatures as plf
from sklearn.ensemble import GradientBoostingClassifier as GBDT
from sklearn import datasets
from gplearn.genetic import SymbolicTransformer

In [29]:
data = np.loadtxt('..\dataset\data1.txt')        # 使用numpy读取数据
data_re = pd.DataFrame(data)                     # 将数据转换为DataFrame
x, y = data_re.iloc[:, :-1], data_re.iloc[:, -1] # 分割数据，得到输入变量x和输出变量y
data_re

,0,1,2,3,4,5,6,7,8,9,10
0,1.886230,1.317859,-0.164806,0.565369,-1.119345,-0.532190,-0.684310,1.241498,1.005792,0.454850,0.0
1,0.450163,0.670809,-1.165714,1.166539,3.276056,-0.872706,-0.310676,-0.949465,-0.331942,-2.943994,1.0
2,0.481587,0.335247,0.722109,-2.017945,-0.425526,-0.980505,1.570869,1.469196,-1.683878,1.449332,1.0
3,-0.701114,-0.528190,0.056773,1.143343,2.916560,0.245130,-0.362534,1.391917,-0.697423,-2.680853,1.0
4,0.715570,1.138827,-0.387145,0.506031,-1.916972,-1.217661,-0.792914,-0.677494,0.550318,1.043190,0.0
...,...,...,...,...,...,...,...,...,...,...,...
995,0.953560,1.000924,-1.826659,-0.938643,-0.063160,0.531305,0.757259,1.248813,-1.648303,0.580475,1.0
996,-0.309807,-1.691897,-1.713937,-2.325080,0.276768,-1.311895,1.961194,1.808892,-0.095440,1.136750,1.0
997,0.361014,-1.103830,1.265656,-0.525260,2.074481,-0.425352,0.839737,0.964460,2.617705,-1.141681,1.0
998,0.886951,0.758980,-1.161209,2.014530,0.532170,1.096608,-1.547043,1.115084,-1.577916,-1.521508,0.0


In [25]:
#使用sklea的feature_selection做特征选择，以下将尝试多种方法
# 采用SelectPercentile选择特征，选取样本总体30%的特征，然后做训练与数据转换
selector_1 = feature_selection.SelectPercentile(percentile=30)
sel_features1 = selector_1.fit_transform(x, y)
# print(sel_features1.shape)
# sel_features1[:5]    # 将1000x10的数据集转换为1000x3的数据集，保留其中的3个特征

# 采用VarianceThreshold选择特征
selector_2 = feature_selection.VarianceThreshold(1)    # 设置阈值1，将方差高于1的特征保留下来
sel_features2 = selector_2.fit_transform(x)    
# print(sel_features2.shape)    # 原始特征中有7个特征符合条件
# sel_features2[:5]

# 采用RFE选择特征
model_svc = SVC(kernel='linear')    # 指定模型分类器，选择SVC，设置线性内核
selector_3 = feature_selection.RFE(model_svc)
sel_features3 = selector_3.fit_transform(x, y)
# print(sel_features3.shape)    # 原始特征中有5个特征符合条件
# sel_features3[:5]

# 采用SelectFromModel选择特征
model_tree = DecisionTreeClassifier(random_state=0)    # 建立分类决策树模型对象
selector_4 = feature_selection.SelectFromModel(model_tree)    # 由于不确定特征重要性的分布，此处不设置特征重要性过滤阈值
sel_features4 = selector_4.fit_transform(x, y)
# print(sel_features4.shape)    # 原始特征中有3个特征符合条件
# sel_features4[:5]

**LDA转换后特征数量小于等于目标变量Label唯一值的个数**

假设原始数据中的目标变量y的唯一值数量有n个，那么LDA转换后可指定的特征个数最多为n-1. 例如set(y)可以看到y的唯一值域只有2个，故LDA的成分最多只能有1个.那么其成分的解释方差比例就是100%，若是多个成分，会展现每个成分的解释方差比例，其和为1.

In [26]:
# 采用LDA进行维度转换
model_lda = LDA()    # 建立LDA模型对象
model_lda.fit(x, y)
convert_features = model_lda.transform(x)
# print(convert_features.shape)    # LDA转换后只有1个特征，该特征不是原有特征，而是转换后的新特征
# print(model_lda.explained_variance_ratio_)    # 获得各成分解释方差占比
# convert_features[:5]
print(set(y))

{0.0, 1.0}


D:\Anaconda\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


**GBDT的组合特征是如何产生的**

GBDT是由大量tree(默认是CART)的集成，假设在GBDT的参数中设置了小树的个数为3(n_estimators=3)，深度为2(max_depth=2)，在GBDT模型训练之后，
就会得到一个基本的GBDT模型；参数n_estimators可自行设置其值，最终特征数量等于n_estimators的树的个数

In [28]:
 # 采用sklearn的GBDT方法组合特征
model_gbdt = GBDT()
model_gbdt.fit(x, y)
conbine_features = model_gbdt.apply(x)[:, :, 0]    # 对原始数据做特征提取
# GBDT提取特征后的索引节点的形状，新的组合特征是100，已经根据最佳分裂节点做离散化处理，后续可以接其他模型做处理
print(conbine_features.shape)    
conbine_features[0]

(1000, 100)


array([ 4.,  4.,  4.,  3.,  5.,  5.,  5.,  4.,  4.,  7.,  7.,  7.,  4.,
        7.,  7.,  4.,  7.,  6., 13., 13., 13., 13.,  7., 13., 13.,  7.,
       13.,  7., 14.,  7., 12.,  6., 13.,  7., 11.,  3., 14.,  4., 13.,
       14.,  4., 11., 13.,  8., 12., 14.,  4., 14.,  3.,  8., 14.,  4.,
        6., 12., 10.,  8.,  3., 13., 14.,  9., 10., 10.,  7., 11., 11.,
        4., 10., 10., 11.,  3.,  3.,  7.,  9., 10., 14., 13., 11.,  4.,
        6., 11., 11.,  6., 14., 14., 10.,  4., 11., 11.,  6., 14.,  6.,
       14., 13.,  8., 10., 12., 13.,  4., 11., 14.])

**PolynomialFeatures(多项式特征组合)方法组合特征**

多项式可以完美的拟合出训练集的数据特征，可以通过设置degree的值来指定多项式的项数，但过高的项数可能会造成过拟合，过低的项数可能会导致欠拟合，所以确定最佳的degree值是重点和难点，在二维空间下，一般通过散点图观察数据的分布规律，在高维下。这几乎是不可能的

In [37]:
# 采用sklearn的PolynomialFeatures方法组合特征
model_plf = plf(2)    # 多项式的项数为2
plf_features = model_plf.fit_transform(x, y)
print(plf_features.shape)
# plf_features[0]
# plf_features[:, 0]
# plf_features[:, 1:9]

(1000, 66)


#### 使用gplearn的genetic方法组合特征
**遗传算法gplearn主要包含两个处理技术**
- SymbolicRegressor(符号回归)，旨在识别最能描述关系的基础数学表达式，它首先建立一个简单随机公式的数量来表示已知自变量与其因变量目标之间的关系，以预测新数据
- SymbolicTransformer(符号转换器)是已知监督式的特征处理技术，首先建立一组简单的随机公式来表示关系，然后通过从群体中选择最合适的个体进行遗传操作，最终找出最适合彼此、相关性最小的个体

In [43]:
# 采用gplearn的genetic方法组合特征
raw_data = datasets.load_boston()
x, y = raw_data.data, raw_data.target
print(x.shape)    # 数据包含506条记录，13个特征
x[0]

# 使用SymbolicTransformer方法进行组合，组合后的特征为5个，迭代次数为18
model_symbolic = SymbolicTransformer(n_components=5, generations=18,
                                     function_set=(
                                         'add', 'sub', 'mul', 'div', 'sqrt', 'log', 'abs', 'neg',
                                         'inv','max', 'min'),
                                     max_samples=0.9, metric='pearson',
                                     random_state=0, n_jobs=2)
model_symbolic.fit(x, y)
symbolic_features = model_symbolic.transform(x)
print(symbolic_features.shape)
symbolic_features[0]

(506, 13)
(506, 5)


array([ 1.2132401 ,  0.83252613, -1.84617541,  0.11456174,  0.48038727])

In [45]:
print(model_symbolic)    # 最后生成的5个特征主要是基于第11和13个原始特征，做复杂的公式组合生成的

[inv(sqrt(inv(sqrt(log(sqrt(inv(mul(X10, X12)))))))),
 sqrt(inv(sqrt(sqrt(log(mul(X10, X12)))))),
 inv(log(inv(sqrt(sqrt(sqrt(mul(X10, X12))))))),
 inv(sqrt(mul(X10, X12))),
 inv(sqrt(log(mul(X10, X12))))]
